In [11]:
import pandas as pd
import folium
import qrcode

# 파일 경로 설정
filePath = r'지역별 폭염일수.xlsx'
df_from_excel = pd.read_excel(filePath, engine='openpyxl', header=None)
df_from_excel.columns = ['이름', '위도', '경도', '폭염일수']

# 위도와 경도를 숫자로 변환 (변환 불가능한 값은 NaN 처리)
df_from_excel['위도'] = pd.to_numeric(df_from_excel['위도'], errors='coerce')
df_from_excel['경도'] = pd.to_numeric(df_from_excel['경도'], errors='coerce')

# NaN 값을 가진 행은 제외
df_from_excel = df_from_excel.dropna(subset=['위도', '경도'])

# 지도 생성
map = folium.Map(location=[37, 127], zoom_start=7)

# 마커 추가
for i in range(len(df_from_excel)):
    # iloc을 사용하여 인덱스 번호로 접근
    x, y, name = df_from_excel.iloc[i][['위도', '경도', '이름']]
    if x != 0 and y != 0:  # 좌표가 유효한 경우
        marker = folium.Marker(
            location=[y, x],
            popup=name,
            icon=folium.Icon(color='blue')
        )
        marker.add_to(map)

# HTML 파일로 저장
html_filename = 'heartwave_map.html'
map.save(html_filename)

# QR 코드 생성
qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=10,
    border=4,
)
url = f'file:///{html_filename}'  # 로컬 파일 경로
qr.add_data(url)
qr.make(fit=True)

# QR 코드 이미지 저장
qr_filename = 'heartwave_qr.png'
qr_image = qr.make_image(fill='black', back_color='white')
qr_image.save(qr_filename)

# QR 코드 확인 (옵션)
qr_image.show()
